#### Set environment variables in [.env](.env) for LLM API calling

### Import Dependencies

In [13]:
import sys
sys.path.insert(0, "../../")
import promptwizard
from promptwizard.glue.promptopt.instantiate import GluePromptOpt
from promptwizard.glue.promptopt.techniques.common_logic import DatasetSpecificProcessing
from promptwizard.glue.common.utils.file import save_jsonlist
from typing import Any
from tqdm import tqdm
from re import compile, findall
import os
from datasets import load_dataset

import random
import json

from dotenv import load_dotenv
load_dotenv(override = True)

True

### Create a dataset specific class and define the required functions 

In [ ]:
# dataset = load_dataset("clinc_oos", "small")

# test_split = dataset["test"]
# texts = test_split["text"]
# intents = test_split["intent"]

# pairs = list(zip(texts, intents))

# # Shuffle the pairs to ensure randomness
# random.shuffle(pairs)

# # Combine texts and intents into pairs
# pairs = list(zip(texts, intents))

# for dataset in ["train", "test"]
#     for i in range(0, 50):
#         # Randomly sample pairs from the dataset
#         sampled_pairs = random.sample(pairs, random.randint(10,30))

#         # Separate the sampled pairs into texts and intents
#         sampled_texts, sampled_intents = zip(*sampled_pairs)

#         # Create the training dataset
#         training_dataset = {
#             "question": f"{list(sampled_texts)}",
#             "final_answer": f"{list(sampled_intents)}"
#         }
        
#         with open(f"data/{dataset}.jsonl", "a") as train_file:
#             train_file.write(json.dumps(training_dataset) + "\n")

In [14]:
class Clustering(DatasetSpecificProcessing):

    def dataset_to_jsonl(self, dataset_jsonl: str, **kwargs: Any) -> None:
        def extract_answer_from_output(completion):
            # Your functions for metrics and prompt building
            ans_re = compile(r"#### (\-?[0-9\.\,]+)")
            self.INVALID_ANS = "[invalid]"

            match = ans_re.search(completion)
            if match:
                match_str = match.group(1).strip()
                match_str = match_str.replace(",", "")
                return match_str
            else:
                return self.INVALID_ANS

        examples_set = []

        for _, sample in tqdm(enumerate(kwargs["dataset"]), desc="Evaluating samples"):
            example = {
              DatasetSpecificProcessing.QUESTION_LITERAL: sample['question'],
              DatasetSpecificProcessing.ANSWER_WITH_REASON_LITERAL: sample['answer'],
              DatasetSpecificProcessing.FINAL_ANSWER_LITERAL: extract_answer_from_output(sample["answer"])
            }
            examples_set.append(example)

        save_jsonlist(dataset_jsonl, examples_set, "w")

    def extract_final_answer(self, answer: str):
        
        if not answer:
            return self.INVALID_ANS

        model_pred = answer.lower()
        preds = model_pred.split(self.ANSWER_START.lower())

        pred = preds[-1].split(self.ANSWER_END.lower())[0].strip()

        if len(pred) == 0:
            return self.INVALID_ANS

        return pred

In [15]:
gsm8k_processor = Clustering()

### Set paths

In [17]:
train_file_name = os.path.join("data", "train.jsonl")
test_file_name = os.path.join("data", "test.jsonl")
path_to_config = "configs"
promptopt_config_path = os.path.join(path_to_config, "promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

### Create an object for calling prompt optimization and inference functionalities

In [18]:
gp = GluePromptOpt(promptopt_config_path,
                   setup_config_path,
                   train_file_name,
                   gsm8k_processor)

Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'clustering'), ('mode', 'offline')] 


Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'clustering'), ('mode', 'offline')] 


Prompt Optimization parameters: [('answer_format', 'For each sentence, assignment it to one of the {k} cluster label and output the cluster number. Your output should ONLY contain a list of {n} integers in the format <ANS_START>[cluster asignments]<ANS_END>. Do not include any other texts.'), ('base_instruction', 'Avoiding Omission Error is of top priority. You must ensure the final output contains exactly {n} cluster labels, not more and not less.'), ('few_shot_count', 3), ('generate_expert_identity', True), ('generate_intent_key

### Call prompt optmization function
1. ```use_examples``` can be used when there are training samples and a mixture of real and synthetic in-context examples are required in the final prompt. When set to ```False``` all the in-context examples will be real
2. ```generate_synthetic_examples``` can be used when there are no training samples and we want to generate synthetic examples 
3. ```run_without_train_examples``` can be used when there are no training samples and in-context examples are not required in the final prompt 

In [19]:
# Function call to generate optimal prompt and expert profile 
best_prompt, expert_profile = gp.get_best_prompt(use_examples=True,run_without_train_examples=False,generate_synthetic_examples=False)


Mutating Task Description....


Iterations completed:   0%|          | 0/3 [00:00<?, ?it/s]
 + Starting iteration: 1 
 current_base_instruction: Avoiding Omission Error is of top priority. You must ensure the final output contains exactly {n} cluster labels, not more and not less.

 + Starting iteration: 1 
 current_base_instruction: Avoiding Omission Error is of top priority. You must ensure the final output contains exactly {n} cluster labels, not more and not less.




#################### Running 




mutation_round=0 mutated_sample_prompt=You are given a task description and a prompt instruction and different styles known as meta prompts:
[Task Description]: You are given a dataset of {n} sentences which you need to cluster into one of the {k} clusters. Output exactly {n} cluster labels.
[Meta Prompt]: What is the targeted output that I should produce?
How many instances of the output should I produce?
How could I ensure that the output is correct and complete with the expected number of response?
How can I break down this problem into smaller pieces so that the response format is guaranteed to be correct?
Try creative thinking, allocating instances into clusters one-by-one can lead to a complete list of labels with all instances covered.
Now you need to generate 5 variations of following Instruction adaptively mixing meta prompt while keeping similar semantic meaning.
Make sure to wrap each generated prompt with <START> and <END>
[Prompt Instruction]: Avoiding Omission Error is of



#################### Running 




...

100%|██████████| 3/3 [00:24<00:00,  8.30s/it]



Generating Expert Identity....


#################### Running 




Expert Identity: You are a data scientist with a strong background in machine learning, particularly in natural language processing and unsupervised learning techniques. You have expertise in text analysis and clustering algorithms, enabling you to effectively group sentences based on their semantic similarities. Your skills include applying methods such as k-means clustering, hierarchical clustering, and advanced techniques like topic modeling or sentence embeddings to create meaningful clusters. You possess a deep understanding of the nuances of language and contextual variations, ensuring that your clustering takes into account both syntactical and semantic content. By leveraging your expertise, you can output exactly {n} cluster labels, ensuring each sentence is accurately classified into one of the {k} clusters. Your analytical prowess ensures that the clustering solution is both interpretable and robust, aligning with the underlying patterns of the data.
Expert Identity: You are 


Generating Intent Keywords....


#################### Running 




Final best prompt: 
You are provided with a dataset containing {n} sentences. Your task is to cluster these sentences into exactly {k} predefined clusters and return {n} cluster labels, ensuring each sentence is assigned one label corresponding to the cluster it belongs to.

1. **Defining Predefined Clusters**: Begin by clearly defining the {k} predefined clusters based on provided guidelines or through careful inference from sentence characteristics. Avoid arbitrary cluster assignments by ensuring thorough familiarity with cluster definitions.

2. **Consistent Output Length**: Ensure that the number of output cluster labels matches the number of input sentences. Carefully cross-check this count before finalizing your output to avoid omission errors. Emphasize the importance of maintaining a strict one-to-one correspondence between input sentences and output labels.

3. **Clustering Approach**: Use a robust clustering method to group sentences based on semantic similarities. Consider A

### Save the optimized prompt and expert profile

In [20]:
import pickle 

if not os.path.exists("results"):
    os.system("mkdir results")
    
with open("results/best_prompt.pkl", 'wb') as f:
    pickle.dump(best_prompt, f)
with open("results/expert_profile.pkl", 'wb') as f:
    pickle.dump(expert_profile, f)

print(f"Best prompt: {best_prompt} \nExpert profile: {expert_profile}")

Best prompt: 
You are provided with a dataset containing {n} sentences. Your task is to cluster these sentences into exactly {k} predefined clusters and return {n} cluster labels, ensuring each sentence is assigned one label corresponding to the cluster it belongs to.

1. **Defining Predefined Clusters**: Begin by clearly defining the {k} predefined clusters based on provided guidelines or through careful inference from sentence characteristics. Avoid arbitrary cluster assignments by ensuring thorough familiarity with cluster definitions.

2. **Consistent Output Length**: Ensure that the number of output cluster labels matches the number of input sentences. Carefully cross-check this count before finalizing your output to avoid omission errors. Emphasize the importance of maintaining a strict one-to-one correspondence between input sentences and output labels.

3. **Clustering Approach**: Use a robust clustering method to group sentences based on semantic similarities. Consider AI mode

In [24]:
expert_profile

'You are a data scientist with a strong background in machine learning, particularly in natural language processing and unsupervised learning techniques. You have expertise in text analysis and clustering algorithms, enabling you to effectively group sentences based on their semantic similarities. Your skills include applying methods such as k-means clustering, hierarchical clustering, and advanced techniques like topic modeling or sentence embeddings to create meaningful clusters. You possess a deep understanding of the nuances of language and contextual variations, ensuring that your clustering takes into account both syntactical and semantic content. By leveraging your expertise, you can output exactly {n} cluster labels, ensuring each sentence is accurately classified into one of the {k} clusters. Your analytical prowess ensures that the clustering solution is both interpretable and robust, aligning with the underlying patterns of the data.'

### Evaluate the optimized prompt

In [22]:
gp.EXPERT_PROFILE = expert_profile
gp.BEST_PROMPT = best_prompt

# Function call to evaluate the prompt
accuracy = gp.evaluate(test_file_name)

print(f"Final Accuracy: {accuracy}")

Evaluation started 

Evaluation started 





#################### Running 




{'accuracy': '0/1 : 0.0%', 'predicted': '[7, 19, 14, 5, 12, 9, 8, 13, 18, 15, 1, 6, 11, 8, 16, 17, 4, 10]', 'actual': '[37, 133, 80, 142, 100, 20, 125, 28, 147, 94, 82, 17, 101, 43, 92, 42, 117, 36]'}
{'accuracy': '0/1 : 0.0%', 'predicted': '[7, 19, 14, 5, 12, 9, 8, 13, 18, 15, 1, 6, 11, 8, 16, 17, 4, 10]', 'actual': '[37, 133, 80, 142, 100, 20, 125, 28, 147, 94, 82, 17, 101, 43, 92, 42, 117, 36]'}




#################### Running 




{'accuracy': '0/2 : 0.0%', 'predicted': '[5, 10, 13, 7, 16, 3, 12, 9, 5, 15, 8, 14, 4, 9, 11, 6, 2, 1]', 'actual': '[42, 109, 42, 42, 105, 145, 34, 110, 42, 53, 136, 111, 125, 142, 32, 10, 42, 95]'}
{'accuracy': '0/2 : 0.0%', 'predicted': '[5, 10, 13, 7, 16, 3, 12, 9, 5, 15, 8, 14, 4, 9, 11, 6, 2, 1]', 'actual': '[42, 109, 42, 42, 105, 145, 34, 110, 42, 53, 136, 111, 125, 142, 32, 10, 42, 95]'}


...



#################### Running 




{'accuracy': '0/49 : 0.0%', 'predicted': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 8, 15, 16, 17, 18, 19, 20, 6]', 'actual': '[42, 145, 21, 81, 17, 58, 141, 123, 34, 85, 112, 94, 126, 123, 28, 54, 102, 35, 32, 11, 58]'}
{'accuracy': '0/49 : 0.0%', 'predicted': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 8, 15, 16, 17, 18, 19, 20, 6]', 'actual': '[42, 145, 21, 81, 17, 58, 141, 123, 34, 85, 112, 94, 126, 123, 28, 54, 102, 35, 32, 11, 58]'}




#################### Running 




{'accuracy': '0/50 : 0.0%', 'predicted': '[38, 72, 15, 57, 84, 33, 29, 41, 95, 67]', 'actual': '[126, 0, 27, 124, 42, 144, 14, 76, 42, 85]'}
{'accuracy': '0/50 : 0.0%', 'predicted': '[38, 72, 15, 57, 84, 33, 29, 41, 95, 67]', 'actual': '[126, 0, 27, 124, 42, 144, 14, 76, 42, 85]'}
Time taken for evaluation: 161.3048689365387 sec
Time taken for evaluation: 161.3048689365387 sec


Final Accuracy: 0.0
